In [ ]:
from rdflib import Graph, URIRef, Namespace, Literal, XSD, RDF
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm.autonotebook import tqdm
import pandas as pd
from copy import deepcopy
import numpy
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
GRAPH_1_INV_FUNC_PATH = ''
GRAPH_1 = ""
GRAPH_2_INV_FUNC_PATH = ''
GRAPH_2 = ""
#GRAPH_2 = "http://localhost:9999/bigdata/sparql"
SUBJECT = 'subject'
PREDICATE = 'predicate'
INVERSE_FUNCTIONALITY = 'inverse_functionality'


In [3]:
graph_1_inv_func_df = pd.read_csv(GRAPH_1_INV_FUNC_PATH)

threshold_graph_1_inv_func_df = graph_1_inv_func_df[graph_1_inv_func_df[INVERSE_FUNCTIONALITY] > 0.5]

#graph_1_predicate_list = threshold_graph_1_inv_func_df[PREDICATE].tolist()

graph_1 = Graph()
graph_1.parse(GRAPH_1)

<Graph identifier=Nd3614b24accd4ca2ad6c098bb1ecf63e (<class 'rdflib.graph.Graph'>)>

In [4]:
literal_y1_dict = dict()
used_p1_dict = dict()
graph_1_predicate_list = list()

for index, row in tqdm(threshold_graph_1_inv_func_df.iterrows(), total=len(threshold_graph_1_inv_func_df)):
    predicate = row[PREDICATE]
    predicate_uri = URIRef(predicate)
    
    if sum(1 for _ in graph_1.subject_objects(predicate_uri)) < 5:
        continue
    
    graph_1_predicate_list.append(predicate)
    
    for s, p, o in graph_1.triples((None, predicate_uri, None)):

        if type(o) != Literal or not isinstance(o, Literal) or not (o.datatype is None or o.datatype == XSD.string or o.datatype == RDF.langString):
            continue
        
        o_value = str(o)
        if o_value not in literal_y1_dict.keys():
            literal_y1_dict[o_value] = list()
        y1_value_list = literal_y1_dict[o_value]
        y1_value_list.append({
            SUBJECT: s,
            PREDICATE: predicate_uri
        })

  0%|          | 0/104 [00:00<?, ?it/s]

In [5]:
graph_2_inv_func_df = pd.read_csv(GRAPH_2_INV_FUNC_PATH)

threshold_graph_2_inv_func_df = graph_2_inv_func_df[graph_2_inv_func_df[INVERSE_FUNCTIONALITY] > 0.5]

#graph_2_predicate_list = threshold_graph_2_inv_func_df[PREDICATE].tolist()

graph_2 = Graph()
graph_2.parse(GRAPH_2)
#graph_2 = Graph("SPARQLStore")
#graph_2.open(GRAPH_2)

<Graph identifier=Ne060f71cbffc47c5b5b60eb4dcd398c6 (<class 'rdflib.graph.Graph'>)>

In [6]:
y2_matches = dict()
graph_2_predicate_list = list()

for index, row in tqdm(threshold_graph_2_inv_func_df.iterrows(), total=len(threshold_graph_2_inv_func_df)):
    predicate = row[PREDICATE]
    predicate_uri = URIRef(predicate)
    
    if sum(1 for _ in graph_2.subject_objects(predicate_uri)) < 5:
        continue
        
    graph_2_predicate_list.append(predicate)
    
    for s, o in graph_2.subject_objects(predicate_uri):
        if type(o) != Literal or not isinstance(o, Literal) or not (o.datatype is None or o.datatype == XSD.string or o.datatype == RDF.langString):
            continue
        
        # Check if exist match
        o_value = str(o)
        if o_value not in literal_y1_dict.keys():
            continue
            
        if o_value not in y2_matches.keys():
            y2_matches[o_value] = list()
        y2_match_list = y2_matches[o_value]
        y2_match_list.append({
            SUBJECT: s,
            PREDICATE: predicate_uri
        })

  0%|          | 0/122 [00:00<?, ?it/s]

In [7]:
def get_predicate_prob(predicate_pair_dict, p1, p2):
    if p1 not in predicate_pair_dict.keys():
        predicate_pair_dict[p1] = dict()
    if p2 not in predicate_pair_dict[p1].keys():
        predicate_pair_dict[p1][p2] = 0.1

    return predicate_pair_dict[p1][p2]

In [8]:
inv_func_1_dict = dict()

for index, row in tqdm(threshold_graph_1_inv_func_df.iterrows(), total=len(threshold_graph_1_inv_func_df)):
    inv_func_1_dict[row['predicate']] = row['inverse_functionality']

  0%|          | 0/104 [00:00<?, ?it/s]

In [9]:
inv_func_2_dict = dict()

for index, row in tqdm(threshold_graph_2_inv_func_df.iterrows(), total=len(threshold_graph_2_inv_func_df)):
    inv_func_2_dict[row['predicate']] = row['inverse_functionality']

  0%|          | 0/122 [00:00<?, ?it/s]

In [10]:
def save_predicate_pair(predicate_pair_counter, x1, p1, y1, x2, p2, y2):
    predicate_pair_counter.append({
        'x1': x1,
        'p1': p1,
        'y1': y1,
        'x2': x2,
        'p2': p2,
        'y2': y2,
    })

In [11]:
def update_prob_entities(entity_pair_match_dict, x1, x2, p1, p2, prob_y):
    if x1 not in entity_pair_match_dict.keys():
        entity_pair_match_dict[x1] = dict()
    if x2 not in entity_pair_match_dict[x1].keys():
        entity_pair_match_dict[x1][x2] = 1
    #if x1 == 'http://dbkwik.webdatacommons.org/finalfantasy./resource/Toad_(Final_Fantasy_V_enemy)':
    #    print(len(entity_pair_match_dict[x1]))
    
    y1_pred = str(p1)
    y2_pred = str(p2)
    pred_prob_1_2 = get_predicate_prob(predicate_pair_dict_1, y1_pred, y2_pred)
    pred_prob_2_1 = get_predicate_prob(predicate_pair_dict_2, y2_pred, y1_pred)

    new_factor = (1-pred_prob_1_2*inv_func_1_dict[y1_pred] * prob_y) * (1-pred_prob_2_1*inv_func_2_dict[y2_pred] * prob_y)

    entity_pair_match_dict[x1][x2] *= new_factor

    return 1 - entity_pair_match_dict[x1][x2]

In [12]:
def calculate_entity_prob_using_attribute(new_entity_pair_match_dict):
    for o_value, y2_match_list in tqdm(y2_matches.items(), desc='Attribute loop', leave=False):
        y1_match_list = literal_y1_dict[o_value]

        for y1_match in y1_match_list:
            y1_subject = y1_match['subject']

            for y2_match in y2_match_list:
                y2_subject = y2_match['subject']


                y1_pred = str(y1_match['predicate'])
                y2_pred = str(y2_match['predicate'])
                sim = update_prob_entities(new_entity_pair_match_dict, str(y1_subject), str(y2_subject),
                                     y1_pred, y2_pred, 1)

                if sim > 0.1:
                    save_predicate_pair(predicate_pair_counter, str(y1_subject), y1_pred, o_value, str(y2_subject),
                                        y2_pred, o_value)
                

In [13]:
def get_inbound_neighbors(graph, predicate_list, y, loaded_neighbors_dict):
    if not y:
        return []
    
    if y in loaded_neighbors_dict.keys():
        return loaded_neighbors_dict[y]
    
    y_url = y
    if type(y_url) != URIRef:
        y_url = URIRef(y)
    
    attempts = 0
    while attempts < 10:
        try:
            neighbor_list = list()
            for s, p in graph.subject_predicates(y_url):
                if str(p) in predicate_list:
                    neighbor_list.append({
                        'p': p,
                        's': s
                    })
            return neighbor_list
        except Exception as e:
            print(e)
            attempts += 1
    loaded_neighbors_dict[y] = neighbor_list
    return neighbor_list

In [14]:
def update_prob_entity_neighbors(entity_pair_dict, y1_neighbors,
                                 y2_neighbors, y_sim, y1, y2):
    for x1 in y1_neighbors:
        for x2 in y2_neighbors:
            x1_str = str(x1['s'])
            x2_str = str(x2['s'])
            p1_str = str(x1['p'])
            p2_str = str(x2['p'])
            sim = update_prob_entities(entity_pair_dict, x1_str, x2_str, p1_str, str(x2['p']),
                                 y_sim)
            if sim > 0.1:
                save_predicate_pair(predicate_pair_counter, x1_str, p1_str, str(y1), x2_str, p2_str, str(y2))

In [32]:
def calculate_entity_prob_using_neighbors(new_entity_pair_dict, old_entity_pair_dict, loaded_neighbors_dict):
    y1_keys = list(old_entity_pair_dict.keys())
    for y1 in tqdm(y1_keys, desc='neighbor loop', leave=False):
        y1_neighbors = get_inbound_neighbors(graph_1, graph_1_predicate_list, y1, loaded_neighbors_dict)
        y2_keys = list(old_entity_pair_dict[y1].keys())
        for y2 in y2_keys:
            y_sim = 1
            if y1 in new_entity_pair_dict.keys() and y2 in new_entity_pair_dict[y1].keys():
                y_sim -= new_entity_pair_dict[y1][y2]
            else:
                y_sim -= old_entity_pair_dict[y1][y2]
            if y_sim < 0.2:
                continue
            
            y2_neighbors = get_inbound_neighbors(graph_2, graph_2_predicate_list, y2, loaded_neighbors_dict)
            update_prob_entity_neighbors(new_entity_pair_dict, y1_neighbors, y2_neighbors, y_sim, y1, y2)
    

In [16]:
def get_prob_entity_pair(entity_pair_match_dict, s1, s2):
    if s1 in entity_pair_match_dict.keys() and s2 in entity_pair_match_dict[s1].keys():
        return 1 - entity_pair_match_dict[s1][s2]
    return 0

In [17]:
def get_set_of_triples_by_predicate(predicate_pair_counter_df, reversed_bool):
    if reversed_bool:
        return predicate_pair_counter_df[['x2', 'p2', 'y2']].drop_duplicates()
    else:
        return predicate_pair_counter_df[['x1', 'p1', 'y1']].drop_duplicates()

In [18]:
def get_columns(reversed_bool):
    if reversed_bool:
        return {
            'x1': 'x2',
            'p1': 'p2',
            'y1': 'y2',
            'x2': 'x1',
            'p2': 'p1',
            'y2': 'y1'
        }
    else:
        return {
            'x1': 'x1',
            'p1': 'p1',
            'y1': 'y1',
            'x2': 'x2',
            'p2': 'p2',
            'y2': 'y2'
        }

In [19]:
def calculate_product(matched_row, entity_pair_match_dict):
    x_sim = 1 - entity_pair_match_dict[matched_row['x1']][matched_row['x2']]
    if matched_row['y1'] == matched_row['y2']:
        y_sim = 1
    else:
        y_sim = entity_pair_match_dict[matched_row['y1']][matched_row['y2']]
    return 1 - x_sim * y_sim

In [20]:
def calculate_sum_by_triple(indexed_df, triple_row):
    matched_triples = indexed_df.join(triple_row)
    return 1.0 - numpy.product(matched_triples['product_element'])

In [21]:
def calculate_upper_sum(entity_pair_match_dict, predicate_pair_counter_df, unique_triples, p1, p2, reversed_bool):
    columns = get_columns(reversed_bool)
    
    filtered_by_ps_df = predicate_pair_counter_df[predicate_pair_counter_df[columns['p2']].isin([p2])]
    
    grouped_df = filtered_by_ps_df.groupby([columns['x1'], columns['y1']])
    product = grouped_df['product_element'].prod()
    return (1 - product).sum()

In [22]:
def calculate_lower_sum(entity_pair_match_dict, predicate_pair_counter_df, unique_triples, p1, reversed_bool):
    columns = get_columns(reversed_bool)
    
    grouped_df = predicate_pair_counter_df.groupby([columns['x1'], columns['y1']])
    product = grouped_df['product_element'].prod()
    
    return (1 - product).sum()

In [23]:
def calculate_x_sim(x1, x2, entity_pair_match_dict, reversed_bool):
    if reversed_bool:
        return 1 - entity_pair_match_dict[x2][x1]
    return 1 - entity_pair_match_dict[x1][x2]

In [24]:
def calculate_y_sim(y1, y2, entity_pair_match_dict, reversed_bool):
    if y1 == y2:
        return 1
    else:
        if reversed_bool:
            if y2 in entity_pair_match_dict.keys() and y1 in entity_pair_match_dict[y2].keys():
                return 1 - entity_pair_match_dict[y2][y1]
        if y1 in entity_pair_match_dict.keys() and y2 in entity_pair_match_dict[y1].keys():
            return 1 - entity_pair_match_dict[y1][y2]
    return 0

In [25]:
def calculate_prob_predicates(predicate_pair_dict, predicate_pair_counter, entity_pair_match_dict, reversed_bool):
    columns = get_columns(reversed_bool)
    predicate_pair_counter_df = pd.DataFrame(predicate_pair_counter)
    
    predicate_pair_counter_df['x_sim'] = predicate_pair_counter_df.apply(lambda x: calculate_x_sim(x[columns["x1"]], x[columns["x2"]], entity_pair_match_dict, reversed_bool), axis=1)
    predicate_pair_counter_df['y_sim'] = predicate_pair_counter_df.apply(lambda x: calculate_y_sim(x[columns["y1"]], x[columns["y2"]], entity_pair_match_dict, reversed_bool), axis=1)

    predicate_pair_counter_df['product_element'] = predicate_pair_counter_df.apply(lambda x: 1.0 - x['x_sim'] * x['y_sim'], axis=1)

    for p1 in tqdm(predicate_pair_dict.keys(), desc='Predicate loop', leave=False):
        filtered_counter = predicate_pair_counter_df[predicate_pair_counter_df[columns['p1']].isin([p1])]
        if len(filtered_counter) < 10:
            for p2 in predicate_pair_dict[p1].keys():
                predicate_pair_dict[p1][p2] = 0.0
        unique_triples = get_set_of_triples_by_predicate(filtered_counter, reversed_bool)
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair_dict[p1][p2] = calculate_upper_sum(entity_pair_match_dict, filtered_counter, unique_triples, p1,
                                                              p2, reversed_bool)
        
        lower_sum = calculate_lower_sum(entity_pair_match_dict, filtered_counter, unique_triples, p1, reversed_bool)
        if lower_sum == 0.0:
            continue
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair_dict[p1][p2] /= lower_sum
            
            

In [26]:
def get_subject_object(graph, predicate):
    attempts = 0
    while attempts < 10:
        try:
            s_o_list = list()
            for s, o in graph.subject_objects(URIRef(predicate)):
                s_o_list.append({
                        'subject': s,
                        'object': o
                    })
            return s_o_list
        except Exception as e:
            print(e)
            attempts += 1
    
    return s_o_list

In [27]:
def check_if_string(y):
    return type(y) == Literal and y.value and type(y.value) == str

In [28]:
def calculate_sub_pred_product(entity_pair_match_dict, x1, x2, y1, y2):
    if check_if_string(y1) and check_if_string(y2):
        if y1.value != y2.value:
            return 0.0
        y_value = y1.value
        if y_value not in literal_y1_dict.keys() or y_value not in y2_matches.keys():
            return 0.0
        return entity_pair_match_dict[str(x1)][str(x2)]
    return 0.0

In [29]:
def save_entity_pairs(i, entity_pair_match_dict):
    entity_pair_sim_list = list()
    for y1 in entity_pair_match_dict.keys():
        for y2 in entity_pair_match_dict[y1].keys():
            similarity = 1 - entity_pair_match_dict[y1][y2]
            entity_pair_sim_list.append({
                "e1": y1,
                "e2": y2,
                "sim": similarity
            })
    entity_sim_df = pd.DataFrame(entity_pair_sim_list)
    entity_sim_df.to_csv("mcu-marvel/entity_sim_" + str(i) + ".csv")

In [30]:
def save_predicate_pairs_into_df(i, predicate_pair_dict, name):
    predicate_pair_list = list()
    for p1 in predicate_pair_dict.keys():
        for p2 in predicate_pair_dict[p1].keys():
            predicate_pair_list.append({
                "p1": p1,
                "p2": p2,
                "sim": predicate_pair_dict[p1][p2]
            })
    predicate_pair_df = pd.DataFrame(predicate_pair_list)
    predicate_pair_df.to_csv(name + str(i) + ".csv")

In [33]:
old_entity_pair_match_dict = dict()
predicate_pair_dict_1 = dict()
predicate_pair_dict_2 = dict()
loaded_neighbors_dict = dict()

for i in tqdm(range(10), desc='Main loop'):
    predicate_pair_counter = list()

    new_entity_pair_match_dict = dict()
    
    calculate_entity_prob_using_attribute(new_entity_pair_match_dict)     
    
    calculate_entity_prob_using_neighbors(new_entity_pair_match_dict, old_entity_pair_match_dict, loaded_neighbors_dict)

    #calculate_prob_predicates(predicate_pair_dict, new_entity_pair_match_dict)
    calculate_prob_predicates(predicate_pair_dict_1, predicate_pair_counter, new_entity_pair_match_dict, False)
    calculate_prob_predicates(predicate_pair_dict_2, predicate_pair_counter, new_entity_pair_match_dict, True)
    
    save_entity_pairs(i, new_entity_pair_match_dict)
    save_predicate_pairs_into_df(i, predicate_pair_dict_1, "mcu-marvel/predicate_pair_1_")
    save_predicate_pairs_into_df(i, predicate_pair_dict_2, "mcu-marvel/predicate_pair_2_")
    old_entity_pair_match_dict = new_entity_pair_match_dict

Main loop:   0%|          | 0/10 [00:00<?, ?it/s]

Attribute loop:   0%|          | 0/8072 [00:00<?, ?it/s]

neighbor loop: 0it [00:00, ?it/s]

Predicate loop:   0%|          | 0/34 [00:00<?, ?it/s]

P2 loop:   0%|          | 0/23 [00:00<?, ?it/s]

...

P2 loop:   0%|          | 0/41 [00:00<?, ?it/s]

P2 loop:   0%|          | 0/31 [00:00<?, ?it/s]

P2 loop:   0%|          | 0/38 [00:00<?, ?it/s]

P2 loop:   0%|          | 0/45 [00:00<?, ?it/s]

P2 loop:   0%|          | 0/41 [00:00<?, ?it/s]